# Push Docker Image to AWS Elastic Container Registry

In [22]:
!cd container; ./build_and_push.sh drug-prediction-gcn

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  165.9kB
Step 1/13 : ARG REGION=us-west-2
Step 2/13 : FROM continuumio/miniconda3
 ---> 406f2b43ea59
Step 3/13 : RUN apt-get update
 ---> Using cache
 ---> 01b536fe282c
Step 4/13 : RUN apt-get install -y build-essential
 ---> Using cache
 ---> 7a497c4b78b1
Step 5/13 : RUN conda install -y -c conda-forge rdkit==2018.09.3
 ---> Using cache
 ---> ac7f0adc29fc
Step 6/13 : RUN conda install pytorch-cpu torchvision -y -c pytorch
 ---> Using cache
 ---> 895cb85379f1
Step 7/13 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 836f6e41614f
Step 8/13 : RUN conda install -y -c conda-forge matplotlib
 ---> Using cache
 ---> 506e420b1ba2


# Run Local Training Job

In [ ]:
# Download the dataset to local

# download pretrained mol2vec model
!wget -O data/model_300dim.pkl https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl model_300dim.pkl   

# download example data     
!wget -O data/ames.sdf https://github.com/samoturk/mol2vec/raw/master/examples/data/ames.sdf
    
#download hiv data
!wget -O data/hiv.zip https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/molnet_publish/hiv.zip
!cd data; unzip hiv.zip

In [23]:
from sagemaker import get_execution_role

role = get_execution_role()

In [24]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [25]:
from sagemaker.estimator import Estimator

hyperparameters = {'dev-mode':True, 
                   'epochs': 200,
                   'learning-rate':1e-4,
                   'gcn-hidden-feats':64,
                   'classifier-hidden-feats':64,
                   'batch-size':200,
                   'num-hidden-layers':2}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction-gcn:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/graph_neural_net/data')

Creating tmpn_ot_fnx_algo-1-8fghr_1 ... 
Attaching to tmpn_ot_fnx_algo-1-8fghr_12mdone
algo-1-8fghr_1  | 2020-01-28 00:19:37,158 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-8fghr_1  | 2020-01-28 00:19:37,159 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-8fghr_1  | Returning the value itself
algo-1-8fghr_1  | 2020-01-28 00:19:37,172 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-8fghr_1  | Returning the value itself
algo-1-8fghr_1  | 2020-01-28 00:19:37,174 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-8fghr_1  | 2020-01-28 00:19:37,187 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-8fghr_1  | Returning the value itself
algo-1-8fghr_1  | 2020-01-28 00:19:37,189 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-8fghr_1  | 2020-01

# Run A Sagemaker Training Job

In [26]:
import sagemaker as sage

sess = sage.Session()

In [27]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction-gcn'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

405787280505.dkr.ecr.us-west-2.amazonaws.com/drug-prediction-gcn:latest


In [ ]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 3*60*60 # train for max of 3 hours

hyperparameters = {'dev-mode':False, 
                   'epochs': 200,
                   'learning-rate':1e-4,
                   'gcn-hidden-feats':64,
                   'classifier-hidden-feats':64,
                   'batch-size':200,
                   'num-hidden-layers':2}

metric_definitions = [{"Name": "train:loss", 
                       "Regex": "train_loss:(\d+.\d+)"}, 
                      {"Name": "test:auc_roc", 
                       "Regex": "test_score:(\d+.\d+)"},
                      {"Name": "val:auc_roc", 
                       "Regex": "auc_roc:(\d+.\d+)"},
                      {"Name": "val: f1_score", 
                       "Regex": "f1_score:(\d+.\d+)"}
                     ]

instance_type = 'ml.m5.4xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                       train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time,
                     metric_definitions=metric_definitions)

estimator.fit(data_location)


2020-01-28 00:20:01 Starting - Starting the training job...